In [116]:
data_path = 'C:/users/juwon/practice_aiffel/gd_nlp/seq2seq/'
kor_path = data_path + 'korean-english-park.train.ko'
eng_path = data_path + 'korean-english-park.train.en'

def clean_corpsus(kor_path, en_path):
    with open(kor_path, 'r', encoding ='UTF-8') as f : kor = f.read().splitlines()
    with open(eng_path, 'r', encoding ='UTF-8') as f : eng = f.read().splitlines()
    
    return kor, eng

kor , eng = clean_corpsus(kor_path, eng_path)

In [117]:
import pandas as pd
data = pd.DataFrame({'ko':kor, 'en': eng})
data

,ko,en
0,"개인용 컴퓨터 사용의 상당 부분은 ""이것보다 뛰어날 수 있느냐?""","Much of personal computing is about ""can you t..."
1,모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하...,so a mention a few weeks ago about a rechargea...
2,그러나 이것은 또한 책상도 필요로 하지 않는다.,"Like all optical mice, But it also doesn't nee..."
3,"79.95달러하는 이 최첨단 무선 광마우스는 허공에서 팔목, 팔, 그외에 어떤 부분...",uses gyroscopic sensors to control the cursor ...
4,정보 관리들은 동남 아시아에서의 선박들에 대한 많은 (테러) 계획들이 실패로 돌아갔...,Intelligence officials have revealed a spate o...
...,...,...
94118,“우리는 3월 8일 김승연 회장과 그의 아들이 보복폭행에 가담한 혐의를 찾기 위해 ...,””We are hoping to seize material evidence to ...
94119,월요일 술집 종업원 6명은 김회장과 아들에게 폭행을 당했음을 진술했다고 경찰은 말했다.,"” On Monday, police secured statements from si..."
94120,그러나 불충분한 증거 확보로 수사에 어려움이 있다.,But the lack of material evidence is making it...
94121,김회장과 그의 아들은 보복폭행 혐의를 강력히 부인하고 있다.,Kim and his son both deny the allegations.


In [118]:
data.drop_duplicates(inplace = True, ignore_index = True)
data.dropna(how='any',axis =0,inplace=True)

In [119]:
data

,ko,en
0,"개인용 컴퓨터 사용의 상당 부분은 ""이것보다 뛰어날 수 있느냐?""","Much of personal computing is about ""can you t..."
1,모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하...,so a mention a few weeks ago about a rechargea...
2,그러나 이것은 또한 책상도 필요로 하지 않는다.,"Like all optical mice, But it also doesn't nee..."
3,"79.95달러하는 이 최첨단 무선 광마우스는 허공에서 팔목, 팔, 그외에 어떤 부분...",uses gyroscopic sensors to control the cursor ...
4,정보 관리들은 동남 아시아에서의 선박들에 대한 많은 (테러) 계획들이 실패로 돌아갔...,Intelligence officials have revealed a spate o...
...,...,...
78963,“우리는 3월 8일 김승연 회장과 그의 아들이 보복폭행에 가담한 혐의를 찾기 위해 ...,””We are hoping to seize material evidence to ...
78964,월요일 술집 종업원 6명은 김회장과 아들에게 폭행을 당했음을 진술했다고 경찰은 말했다.,"” On Monday, police secured statements from si..."
78965,그러나 불충분한 증거 확보로 수사에 어려움이 있다.,But the lack of material evidence is making it...
78966,김회장과 그의 아들은 보복폭행 혐의를 강력히 부인하고 있다.,Kim and his son both deny the allegations.


In [120]:
import re
def preprocessing_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'\'+', '\'', sentence)
    sentence = re.sub(r'\"+', '\"', sentence )
    sentence = re.sub(r'\([^)]*\)', r'', sentence) # 괄호안 단어 삭제
    sentence = re.sub(r'[^a-zA-Z가-힣.,?!\'\"]', r' ', sentence)
    sentence = re.sub(r'([.,?!\'\"])', r' \1 ', sentence)
    sentence = re.sub(r'[ ]+', r' ', sentence)
    sentence = sentence.strip()
    return sentence

In [121]:
preprocessing_sentence(data['en'].loc[78964])

'on monday , police secured statements from six victims who said they were beaten directly by kim and his son .'

In [122]:
kor= []
eng = []
for i, sen in enumerate(data['ko']):
    if len(sen) > 15:
        kor.append(preprocessing_sentence(sen))
        eng.append(preprocessing_sentence(data['en'].loc[i]))
print(len(kor))
print(len(eng))

77554
77554


In [123]:
import sentencepiece as spm

data_path = 'C:/users/juwon/practice_aiffel/gd_nlp/transformer/'

def generate_tokenizer(corpus, 
                       vocab_size, 
                       lang='dsf',
                       pad_id=0,
                        bos_id=1,
                        eos_id=2,
                        unk_id=3):
    
    temp_file = data_path + '{}.temp'.format(lang)
    
    with open(temp_file, 'w', encoding='UTF-8') as f :
        for row in corpus:
            f.write(row+'\n')
            
    spm.SentencePieceTrainer.Train('--input={} --pad_id={} --bos_id={} --eos_id={} --unk_id={} --model_prefix={} --vocab_size={}'.format(temp_file, pad_id, bos_id, eos_id, unk_id, lang, vocab_size))
    s = spm.SentencePieceProcessor()
    s.Load('{}.model'.format(lang))
    
    return s

kor_vocab_size = eng_vocab_size = 20000
ko_tokenizer = generate_tokenizer(kor, kor_vocab_size, 'ko')
en_tokenizer= generate_tokenizer(eng, eng_vocab_size, 'en')
en_tokenizer.set_encode_extra_options('bos:eos')

True

In [124]:
import tensorflow as tf
import random
src_corpus = []
tgt_corpus = []

for i, sen in enumerate(kor):
    kor_sen = ko_tokenizer.EncodeAsIds(sen)
    eng_sen = en_tokenizer.EncodeAsIds(eng[i])
    if (len(kor_sen)< 50) & (len(eng_sen) < 50) :
        src_corpus.append(kor_sen)
        tgt_corpus.append(eng_sen)
src_corpus = random.sample(src_corpus, 30000)
tgt_corpus = random.sample(tgt_corpus, 30000)
        
enc_train = tf.keras.preprocessing.sequence.pad_sequences(src_corpus, padding='post')
dec_train = tf.keras.preprocessing.sequence.pad_sequences(tgt_corpus, padding='post')

In [142]:
import numpy as np
def positional_encoding(pos, d_model):
    table = np.zeros((pos, d_model))
    for i in range(pos): 
        for j in range(d_model):
            table[i][j] = i / np.power(10000, (2 *(j//2) / d_model))
    sinusoid_table = table       
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])
    
    return sinusoid_table

In [143]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads):
        super(MultiHeadAttention, self).__init__()
        self.d_model =d_model
        self.n_heads = n_heads
        self.depth = d_model // self.n_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)
    
    def scaled_dot_product(self, q, k, v, mask):
        d_k = tf.cast(k.shape[-1], tf.float32)
        
        qk = tf.matmul(q, k, transpose_b = True)
        scaled_qk = qk / tf.math.sqrt(d_k)
        
        if mask is not None: scaled_qk += (mask * -1e9) 

        attentions = tf.nn.softmax(scaled_qk, axis = -1)
        out = tf.matmul(attentions, v)
        
        return out, attentions
    
    def split_heads(self, x):
        batch_size = x.shape[0]
        split_x = tf.reshape(x, (batch_size, -1 , self.n_heads, self.depth))
        split_x = tf.transpose(split_x, perm =[0,2,1,3])
        
        return split_x
    
    def combine_heads(self, x):
        batch_size = x.shape[0]
        combined_x = tf.transpose(x, perm=[0,2,1,3])
        combined_x = tf.reshape(combined_x, (batch_size, -1, self.d_model))
        
        return combined_x
    
    def call(self, q, k, v ,mask):
        wq = self.W_q(q)
        wk = self.W_k(k)
        wv = self.W_v(v)
        
        wq_splits = self.split_heads(wq)
        wk_splits = self.split_heads(wk)
        wv_splits = self.split_heads(wv)
        
        out, attentions = self.scaled_dot_product(wq_splits, wk_splits, wv_splits, mask)
        out = self.combine_heads(out)
        out = self.linear(out)
        
        return out, attentions

In [144]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.w_1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.w_2 = tf.keras.layers.Dense(d_model)
        
    def call(self, x):
        out = self.w_1(x)
        out = self.w_2(out)
        
        return out

## Encoder Layer

In [145]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        
        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)
        
        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon = 1e-6) 
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)
        
        self.dropout = tf.keras.layers.Dropout(dropout)
    
    def call(x, mask):
        # Multi Head Attention
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.dropout(out)
        out += residual
        
        # Positionwise Feed Forward Network 
        residual = out 
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.dropout(out)
        out += residual
        
        return out, enc_attn

## Decoder Layer

In [146]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        
        self.dec_self_attn = MultiHeadAttention(d_model, n_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, n_heads)
        
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)
        
        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)

        self.dropout = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, causality_mask, padding_mask):
        # Masked MultiHead attention
        residual = x 
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.dropout(out)
        out += residual
        
        # Multi Head Attention
        residual = out
        out = self.norm_2(out)
        out, enc_dec_attn = self.enc_dec_attn(out, enc_out, enc_out, causality_mask)
        out = self.dropout(out)
        out += residual
        
        # Positionwise Feed Forward Network
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.dropout(out)
        out += residual
        
        return out, dec_attn, enc_dec_attn

## Encoder

In [147]:
class Encoder(tf.keras.Model):
    def __init__(self,
                 n_layers,
                 d_model,
                 n_heads,
                 d_ff,
                 dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)]
        
    def call(self, x, mask):
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
        
        return out, enc_attns

## Decoder

In [148]:
class Decoder(tf.keras.Model):
    def __init__(self, n_layers, d_model, n_heads, d_ff, dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)]
    
    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x
        dec_attns = list()
        dec_enc_attns = list()
        
        for i in range(self.n_layers):
            out , dec_attn, dec_enc_attn = self.dec_layers[i](out, enc_out, causality_mask, padding_mask)
            dec_attns.append(dec_attn)
            enc_dec_attns.append(dec_enc_attn)
        
        return out, dec_attns, dec_enc_attns

## Transformer

In [149]:
class TransFormer(tf.keras.Model):
    def __init__(self, 
                 n_layers, 
                 d_model, 
                 n_heads, 
                 d_ff, 
                 src_vocab_size, 
                 tgt_vocab_size, 
                 pos_len,
                 dropout, 
                 shared = True):
        super(TransFormer, self).__init__()
        self.d_model = tf.cast(d_model, tf.float32)
        
        self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
        self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)
        
        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.dropout = tf.keras.layers.Dropout(dropout)
        
        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)
        
        self.fc = tf.keras.layers.Dense(tgt_vocab_size)
        
        self.shared = shared
        if shared : self.fc.set_weights(tf.transpose(self.dec_emb.weights))
            
    def embedding(self, emb, x):
        seq_len = x.shape[1]
        out = emb(x)
        if self.shared : out *= tf.math.sqrt(self.d_model)
        
        out += self.pos_encoding[np.newaxis,...][:, :seq_len, :]
        out = self.dropout(out)
        
        return out
    
    def call(self, enc_in ,dec_in, enc_mask, causality_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)
        
        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        dec_out, dec_attns, dec_enc_attns = self.decoder(dec_in, enc_out, causality_mask, dec_mask)
        
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns

## masking

In [150]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_causality_mask(src_len, tgt_len):
    mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
    return tf.cast(mask, tf.float32)

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_padding_mask(tgt)
    
    dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
    dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)
    
    dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
    dec_mask = tf.maximum(dec_mask, dec_causality_mask)
    return enc_mask, dec_enc_mask, dec_mask

## optimizer

In [151]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warm_steps=4000):
        super(LearningRateScheduler, self).__init__()
        self.d_model = d_model
        self.warm_steps = warm_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warm_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)
learning_rate = LearningRateScheduler(512)
optimizer = tf.keras.optimizers.Adam(learning_rate,
                                     beta_1 = 0.9,
                                     beta_2 = 0.98,
                                     epsilon = 1e-9)

## loss_function

In [152]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction ='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_sum(loss_) / tf.reduce_sum(mask)

In [153]:
transformer = TransFormer(n_layers=2,
                         d_model = 512,
                         n_heads = 8,
                         d_ff = 2048,
                         src_vocab_size=kor_vocab_size,
                         tgt_vocab_size=eng_vocab_size,
                         pos_len = 49,
                         dropout = 0.3,
                         shared=True)

## train step

In [154]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    gold = tgt[:, 1:]
    
    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt)
    
    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = model(src, tgt, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions[:, :-1])
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizers.apply_gradients(zip(gradients,model.trainable.variables))
    
    return loss, enc_attns, dec_attns, dec_enc_attns

In [155]:
from tqdm import tqdm

BATCH_SIZE = 128
EPOCHS = 10
loss=[]

for epoch in range(EPOCHS):
    total_loss = 0 
    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)
    
    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns  = train_step(enc_train[idx:idx+BATCH_SIZE], 
                                                                      dec_train[idx:idx+BATCH_SIZE], 
                                                                      transformer, 
                                                                      optimizer)
        
        total_loss += batch_loss
        loss.append((total_loss.numpy() / (batch+1)))
        
        t.set_description_str('Epoch %2d' % (epoch+1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

  0%|                                                                                          | 0/235 [00:00<?, ?it/s]


TypeError: in user code:

    File "C:\Users\juwon\AppData\Local\Temp\ipykernel_17836\804274200.py", line 8, in train_step  *
        predictions, enc_attns, dec_attns, dec_enc_attns = model(src, tgt, enc_mask, dec_enc_mask, dec_mask)
    File "C:\Users\juwon\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\juwon\AppData\Local\Temp\__autograph_generated_file21qdgttt.py", line 12, in tf__call
        (enc_out, enc_attns) = ag__.converted_call(ag__.ld(self).encoder, (ag__.ld(enc_in), ag__.ld(enc_mask)), None, fscope)
    File "C:\Users\juwon\AppData\Local\Temp\__autograph_generated_file1mvswwdo.py", line 27, in tf__call
        ag__.for_stmt(ag__.converted_call(ag__.ld(range), (ag__.ld(self).n_layers,), None, fscope), None, loop_body, get_state, set_state, ('out',), {'iterate_names': 'i'})
    File "C:\Users\juwon\AppData\Local\Temp\__autograph_generated_file1mvswwdo.py", li

In [ ]:
# 번역 생성 함수

def evaluate(sentence, model, src_tokenizer, tgt_tokenizer):
    sentence = preprocess_sentence(sentence)

    pieces = src_tokenizer.encode_as_pieces(sentence)
    tokens = src_tokenizer.encode_as_ids(sentence)

    _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')
    
    ids = []
    output = tf.expand_dims([tgt_tokenizer.bos_id()], 0)
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(_input, 
              output,
              enc_padding_mask,
              combined_mask,
              dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if tgt_tokenizer.eos_id() == predicted_id:
            result = tgt_tokenizer.decode_ids(ids)
            return pieces, result, enc_attns, dec_attns, dec_enc_attns

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = tgt_tokenizer.decode_ids(ids)

    return pieces, result, enc_attns, dec_attns, dec_enc_attns

In [ ]:
examples = [
            "오바마는 대통령이다.",
            "시민들은 도시 속에 산다.",
            "커피는 필요 없다.",
            "일곱 명의 사망자가 발생했다."
]
for example in examples :
    translate(example, transformer, ko_tokenizer, en_tokenizer)

에러의 원인을 찾지 못해서 나머지 코드만 작성하여 제출합니다. enc_in과 mask의 연산에서 오류가 나는 것 같은데 코드상에서 문제점을 못찾겠네요